<a href="https://colab.research.google.com/github/yaceben/StableDiffusion-XL-Simplifie-et-Refiner/blob/main/Simple_SDXL_Refiner_FR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Faites-moi tourner en premier! Initialisation de base

# @markdown Le code est adapté du notebook https://colab.research.google.com/github/woctezuma/stable-diffusion-colab/blob/main/stable_diffusion.ipynb#scrollTo=AUc4QJfE-uR9
%pip install --quiet --upgrade diffusers transformers accelerate invisible_watermark mediapy

import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline, schedulers

use_refiner = True

scheduler_kwargs = {
    "beta_end": 0.012,
    "beta_schedule": "scaled_linear", # one of ["linear", "scaled_linear"]
    "beta_start": 0.00085,
    "interpolation_type": "linear", # one of ["linear", "log_linear"]
    "num_train_timesteps": 1000,
    "prediction_type": "epsilon", # one of ["epsilon", "sample", "v_prediction"]
    "steps_offset": 1,
    "timestep_spacing": "leading", # one of ["linspace", "leading"]
    "trained_betas": None,
    "use_karras_sigmas": False,
}

scheduler = schedulers.EulerDiscreteScheduler(**scheduler_kwargs)

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    scheduler = scheduler,
    )

#pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

if use_refiner:
  refiner = DiffusionPipeline.from_pretrained(
      "stabilityai/stable-diffusion-xl-refiner-1.0",
      text_encoder_2=pipe.text_encoder_2,
      vae=pipe.vae,
      torch_dtype=torch.float16,
      use_safetensors=True,
      variant="fp16",
      scheduler = scheduler,
  )

  #refiner.unet = torch.compile(refiner.unet, mode="reduce-overhead", fullgraph=True)
  refiner = refiner.to("cuda")

  pipe.enable_model_cpu_offload()
else:
  pipe = pipe.to("cuda")

In [ ]:
# @title Prompts (Instructions pour générer l'image)
# @
prompt = "a knitted nyan cat, 8k, ultra realistic" # @param {type:"string", allow-input: true}
negative_prompt = "cartoon" # @param {type:"string", allow-input: true}

nombre_itérations = 100 # @param {type:"slider", min:1, max:1000, step:1}
denoising = 0.8 # @param {type:"slider", min:0, max:1, step:0.01}

force_du_prompt = 7.5 # @param {type: "number"}

seed = random.randint(0, sys.maxsize)

negative_prompt = negative_prompt if negative_prompt != "" else None

images = pipe(
    prompt = prompt,
    negative_prompt = negative_prompt,
    num_inference_steps = nombre_itérations,
    denoising_end = denoising,
    guidance_scale = force_du_prompt,
    output_type = "latent" if use_refiner else "pil",
    generator = torch.Generator("cuda").manual_seed(seed),
    ).images

if use_refiner:
    images = refiner(
        prompt = prompt,
        negative_prompt = negative_prompt,
        num_inference_steps = nombre_itérations,
        denoising_start = denoising,
        image = images,
    ).images

print(f"Prompt:\t{prompt}\nSeed:\t{seed}")
media.show_images(images)
images[0].save("output.jpg")